In [8]:
import numpy as np
import cv2
import tensorflow as tf
from PIL import Image,ImageFont,ImageDraw
import os
import skimage.io
import skimage.color
import skimage.transform
import random
from sklearn import svm
import warnings
warnings.filterwarnings("ignore")

final_score = []
total_score = 0
for seed in range(20):    
    random.seed(seed)
    DATASET_DIR = './data/'
    classes = os.listdir(DATASET_DIR)
    data = []
    for cls in classes:
        files = os.listdir(DATASET_DIR + cls)
        for f in files:
            img = skimage.io.imread(DATASET_DIR + cls + "/" + f)
            #img = skimage.color.rgb2gray(img)
            if cls == "good_sit": 
                y = 0
            else:
                y = 1
            data.append({
                'x': img,
                'y': y
            })
    random.shuffle(data)
    X = [d['x'] for d in data]
    Y = [[d['y']] for d in data]

    # 检测模型
    file_model = "posenet_mobilenet_v1_100_257x257_multi_kpt_stripped.tflite"

    interpreter = tf.lite.Interpreter(model_path=file_model)
    interpreter.allocate_tensors()

    # 获取输入、输出的数据的信息
    input_details = interpreter.get_input_details()
    #print('input_details\n',input_details)
    output_details = interpreter.get_output_details()
    #print('output_details',output_details)


    # 获取PosNet 要求输入图像的高和宽
    height = input_details[0]['shape'][1]
    width = input_details[0]['shape'][2]

    key_data = [[]]
    for img in X:
        # 获取图像帧的尺寸
        imH,imW,_ = np.shape(img)

        # 适当缩放
        img = cv2.resize(img,(int(imW*0.5),int(imH*0.5)))

        # 获取图像帧的尺寸
        imH,imW,_ = np.shape(img)

        # BGR 转RGB
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # 尺寸缩放适应PosNet 网络输入要求
        img_resized = cv2.resize(img_rgb, (width, height))

        # 维度扩张适应网络输入要求
        input_data = np.expand_dims(img_resized, axis=0)

        # 尺度缩放 变为 -1~+1
        input_data = (np.float32(input_data) - 128.0)/128.0

        # 数据输入网络
        interpreter.set_tensor(input_details[0]['index'],input_data)

        # 进行关键点检测
        interpreter.invoke()

        # 获取hotmat
        hotmaps = interpreter.get_tensor(output_details[0]['index'])[0] # Bounding box coordinates of detected objects

        # 获取偏移量
        offsets = interpreter.get_tensor(output_details[1]['index'])[0] # Class index of detected objects

        # 获取hotmat的 宽 高 以及关键的数目
        h_output,w_output, n_KeyPoints= np.shape(hotmaps)

        # 存储关键点的具体位置
        keypoints =[]

        # 关键点的置信度
        score_ = []
        for i in range(11):
            # 遍历每一张hotmap
            hotmap = hotmaps[:,:,i]

            # 获取最大值 和最大值的位置
            max_index = np.where(hotmap==np.max(hotmap))
            max_val   = np.max(hotmap)

            # 获取y，x偏移量 前n_KeyPoints张图是y的偏移 后n_KeyPoints张图是x的偏移
            offset_y = offsets[max_index[0],max_index[1],i]
            offset_x = offsets[max_index[0],max_index[1],i+n_KeyPoints]

            # 计算在posnet输入图像中具体的坐标
            pos_y = max_index[0]/(h_output-1)*height + offset_y
            pos_x = max_index[1]/(w_output-1)*width + offset_x

            # 计算在源图像中的坐标
            pos_y = pos_y/(height-1)*imH
            pos_x = pos_x/(width-1)*imW

            #算置信度
            score = 1.0/(1.0+np.exp(-max_val))

            # 取整获得keypoints的位置
            if score>0.2:
                key_data[-1].append(int(round(pos_x[0])))
                key_data[-1].append(int(round(pos_y[0])))
            else:
                key_data[-1].append(-1)
                key_data[-1].append(-1)
            #score_.append(score)

        #print(keypoints[:11], score_)   
        #key_data.append(keypoints[:22])
        key_data.append([])
    key_data = key_data[:-1]

    X_train = np.array(key_data[:-25])
    Y_train = np.array(Y[:-25])
    X_test  = np.array(key_data[-25:])
    Y_test  = np.array(Y[-25:])

    assert len(X_train) == len(Y_train)
    assert len(X_test)  == len(Y_test)

    svc = svm.SVC(kernel="rbf")
    svc.fit(X_train, Y_train)
    a = svc.score(X_test, Y_test)
    print(a,seed)
    final_score.append((a, seed))
    total_score += a
print(final_score)
print(total_score/20)

0.92 0
0.8 1
0.8 2
0.88 3
0.88 4
0.84 5
0.8 6
0.72 7
0.8 8
0.8 9
0.76 10
0.88 11
0.84 12
0.88 13
0.92 14
0.88 15
0.92 16
0.72 17
0.96 18
0.84 19
[(0.92, 0), (0.8, 1), (0.8, 2), (0.88, 3), (0.88, 4), (0.84, 5), (0.8, 6), (0.72, 7), (0.8, 8), (0.8, 9), (0.76, 10), (0.88, 11), (0.84, 12), (0.88, 13), (0.92, 14), (0.88, 15), (0.92, 16), (0.72, 17), (0.96, 18), (0.84, 19)]
0.8420000000000002


In [7]:
import numpy as np
import cv2
import tensorflow as tf
from PIL import Image,ImageFont,ImageDraw
import os
import skimage.io
import skimage.color
import skimage.transform
import random
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings("ignore")

final_score = []
total_score = 0
for seed in range(20):    
    random.seed(seed)
    DATASET_DIR = './data/'
    classes = os.listdir(DATASET_DIR)
    data = []
    for cls in classes:
        files = os.listdir(DATASET_DIR + cls)
        for f in files:
            img = skimage.io.imread(DATASET_DIR + cls + "/" + f)
            #img = skimage.color.rgb2gray(img)
            if cls == "good_sit": 
                y = 0
            else:
                y = 1
            data.append({
                'x': img,
                'y': y
            })
    random.shuffle(data)
    X = [d['x'] for d in data]
    Y = [[d['y']] for d in data]

    # 检测模型
    file_model = "posenet_mobilenet_v1_100_257x257_multi_kpt_stripped.tflite"

    interpreter = tf.lite.Interpreter(model_path=file_model)
    interpreter.allocate_tensors()

    # 获取输入、输出的数据的信息
    input_details = interpreter.get_input_details()
    #print('input_details\n',input_details)
    output_details = interpreter.get_output_details()
    #print('output_details',output_details)


    # 获取PosNet 要求输入图像的高和宽
    height = input_details[0]['shape'][1]
    width = input_details[0]['shape'][2]

    key_data = [[]]
    for img in X:
        # 获取图像帧的尺寸
        imH,imW,_ = np.shape(img)

        # 适当缩放
        img = cv2.resize(img,(int(imW*0.5),int(imH*0.5)))

        # 获取图像帧的尺寸
        imH,imW,_ = np.shape(img)

        # BGR 转RGB
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # 尺寸缩放适应PosNet 网络输入要求
        img_resized = cv2.resize(img_rgb, (width, height))

        # 维度扩张适应网络输入要求
        input_data = np.expand_dims(img_resized, axis=0)

        # 尺度缩放 变为 -1~+1
        input_data = (np.float32(input_data) - 128.0)/128.0

        # 数据输入网络
        interpreter.set_tensor(input_details[0]['index'],input_data)

        # 进行关键点检测
        interpreter.invoke()

        # 获取hotmat
        hotmaps = interpreter.get_tensor(output_details[0]['index'])[0] # Bounding box coordinates of detected objects

        # 获取偏移量
        offsets = interpreter.get_tensor(output_details[1]['index'])[0] # Class index of detected objects

        # 获取hotmat的 宽 高 以及关键的数目
        h_output,w_output, n_KeyPoints= np.shape(hotmaps)

        # 存储关键点的具体位置
        keypoints =[]

        # 关键点的置信度
        score_ = []
        for i in range(11):
            # 遍历每一张hotmap
            hotmap = hotmaps[:,:,i]

            # 获取最大值 和最大值的位置
            max_index = np.where(hotmap==np.max(hotmap))
            max_val   = np.max(hotmap)

            # 获取y，x偏移量 前n_KeyPoints张图是y的偏移 后n_KeyPoints张图是x的偏移
            offset_y = offsets[max_index[0],max_index[1],i]
            offset_x = offsets[max_index[0],max_index[1],i+n_KeyPoints]

            # 计算在posnet输入图像中具体的坐标
            pos_y = max_index[0]/(h_output-1)*height + offset_y
            pos_x = max_index[1]/(w_output-1)*width + offset_x

            # 计算在源图像中的坐标
            pos_y = pos_y/(height-1)*imH
            pos_x = pos_x/(width-1)*imW

            #算置信度
            score = 1.0/(1.0+np.exp(-max_val))

            # 取整获得keypoints的位置
            if score>0.2:
                key_data[-1].append(int(round(pos_x[0])))
                key_data[-1].append(int(round(pos_y[0])))
            else:
                key_data[-1].append(-1)
                key_data[-1].append(-1)
            #score_.append(score)

        #print(keypoints[:11], score_)   
        #key_data.append(keypoints[:22])
        key_data.append([])
    key_data = key_data[:-1]

    X_train = np.array(key_data[:-25])
    Y_train = np.array(Y[:-25])
    X_test  = np.array(key_data[-25:])
    Y_test  = np.array(Y[-25:])

    assert len(X_train) == len(Y_train)
    assert len(X_test)  == len(Y_test)

    model = LogisticRegression().fit(X_train, Y_train)
    a = model.score(X_test, Y_test)
    print(a,seed)
    final_score.append((a, seed))
    total_score += a
print(final_score)
print(total_score/20)

0.92 0
0.84 1
0.88 2
0.96 3
0.8 4
0.76 5
0.84 6
0.76 7
0.88 8
0.8 9
0.84 10
0.88 11
0.84 12
0.8 13
0.84 14
0.88 15
0.84 16
0.68 17
0.92 18
0.8 19
[(0.92, 0), (0.84, 1), (0.88, 2), (0.96, 3), (0.8, 4), (0.76, 5), (0.84, 6), (0.76, 7), (0.88, 8), (0.8, 9), (0.84, 10), (0.88, 11), (0.84, 12), (0.8, 13), (0.84, 14), (0.88, 15), (0.84, 16), (0.68, 17), (0.92, 18), (0.8, 19)]
0.8380000000000001


In [6]:
import numpy as np
import cv2
import tensorflow as tf
from PIL import Image,ImageFont,ImageDraw
import os
import skimage.io
import skimage.color
import skimage.transform
import random
from sklearn import tree
import warnings
warnings.filterwarnings("ignore")

final_score = []
total_score = 0
for seed in range(20):    
    random.seed(seed)
    DATASET_DIR = './data/'
    classes = os.listdir(DATASET_DIR)
    data = []
    for cls in classes:
        files = os.listdir(DATASET_DIR + cls)
        for f in files:
            img = skimage.io.imread(DATASET_DIR + cls + "/" + f)
            #img = skimage.color.rgb2gray(img)
            if cls == "good_sit": 
                y = 0
            else:
                y = 1
            data.append({
                'x': img,
                'y': y
            })
    random.shuffle(data)
    X = [d['x'] for d in data]
    Y = [[d['y']] for d in data]

    # 检测模型
    file_model = "posenet_mobilenet_v1_100_257x257_multi_kpt_stripped.tflite"

    interpreter = tf.lite.Interpreter(model_path=file_model)
    interpreter.allocate_tensors()

    # 获取输入、输出的数据的信息
    input_details = interpreter.get_input_details()
    #print('input_details\n',input_details)
    output_details = interpreter.get_output_details()
    #print('output_details',output_details)


    # 获取PosNet 要求输入图像的高和宽
    height = input_details[0]['shape'][1]
    width = input_details[0]['shape'][2]

    key_data = [[]]
    for img in X:
        # 获取图像帧的尺寸
        imH,imW,_ = np.shape(img)

        # 适当缩放
        img = cv2.resize(img,(int(imW*0.5),int(imH*0.5)))

        # 获取图像帧的尺寸
        imH,imW,_ = np.shape(img)

        # BGR 转RGB
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # 尺寸缩放适应PosNet 网络输入要求
        img_resized = cv2.resize(img_rgb, (width, height))

        # 维度扩张适应网络输入要求
        input_data = np.expand_dims(img_resized, axis=0)

        # 尺度缩放 变为 -1~+1
        input_data = (np.float32(input_data) - 128.0)/128.0

        # 数据输入网络
        interpreter.set_tensor(input_details[0]['index'],input_data)

        # 进行关键点检测
        interpreter.invoke()

        # 获取hotmat
        hotmaps = interpreter.get_tensor(output_details[0]['index'])[0] # Bounding box coordinates of detected objects

        # 获取偏移量
        offsets = interpreter.get_tensor(output_details[1]['index'])[0] # Class index of detected objects

        # 获取hotmat的 宽 高 以及关键的数目
        h_output,w_output, n_KeyPoints= np.shape(hotmaps)

        # 存储关键点的具体位置
        keypoints =[]

        # 关键点的置信度
        score_ = []
        for i in range(11):
            # 遍历每一张hotmap
            hotmap = hotmaps[:,:,i]

            # 获取最大值 和最大值的位置
            max_index = np.where(hotmap==np.max(hotmap))
            max_val   = np.max(hotmap)

            # 获取y，x偏移量 前n_KeyPoints张图是y的偏移 后n_KeyPoints张图是x的偏移
            offset_y = offsets[max_index[0],max_index[1],i]
            offset_x = offsets[max_index[0],max_index[1],i+n_KeyPoints]

            # 计算在posnet输入图像中具体的坐标
            pos_y = max_index[0]/(h_output-1)*height + offset_y
            pos_x = max_index[1]/(w_output-1)*width + offset_x

            # 计算在源图像中的坐标
            pos_y = pos_y/(height-1)*imH
            pos_x = pos_x/(width-1)*imW

            #算置信度
            score = 1.0/(1.0+np.exp(-max_val))

            # 取整获得keypoints的位置
            if score>0.2:
                key_data[-1].append(int(round(pos_x[0])))
                key_data[-1].append(int(round(pos_y[0])))
            else:
                key_data[-1].append(-1)
                key_data[-1].append(-1)
            #score_.append(score)

        #print(keypoints[:11], score_)   
        #key_data.append(keypoints[:22])
        key_data.append([])
    key_data = key_data[:-1]

    X_train = np.array(key_data[:-25])
    Y_train = np.array(Y[:-25])
    X_test  = np.array(key_data[-25:])
    Y_test  = np.array(Y[-25:])

    assert len(X_train) == len(Y_train)
    assert len(X_test)  == len(Y_test)

    model = tree.DecisionTreeClassifier().fit(X_train, Y_train)
    a = model.score(X_test, Y_test)
    print(a,seed)
    final_score.append((a, seed))
    total_score += a
print(final_score)
print(total_score/20)

0.92 0
0.92 1
0.76 2
0.92 3
0.76 4
0.8 5
0.88 6
0.88 7
0.84 8
0.76 9
0.92 10
0.84 11
0.92 12
0.92 13
0.88 14
0.92 15
0.88 16
0.92 17
0.84 18
0.88 19
[(0.92, 0), (0.92, 1), (0.76, 2), (0.92, 3), (0.76, 4), (0.8, 5), (0.88, 6), (0.88, 7), (0.84, 8), (0.76, 9), (0.92, 10), (0.84, 11), (0.92, 12), (0.92, 13), (0.88, 14), (0.92, 15), (0.88, 16), (0.92, 17), (0.84, 18), (0.88, 19)]
0.868


In [5]:
import numpy as np
import cv2
import tensorflow as tf
from PIL import Image,ImageFont,ImageDraw
import os
import skimage.io
import skimage.color
import skimage.transform
import random
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings("ignore")

final_score = []
total_score = 0
for seed in range(20):    
    random.seed(seed)
    DATASET_DIR = './data/'
    classes = os.listdir(DATASET_DIR)
    data = []
    for cls in classes:
        files = os.listdir(DATASET_DIR + cls)
        for f in files:
            img = skimage.io.imread(DATASET_DIR + cls + "/" + f)
            #img = skimage.color.rgb2gray(img)
            if cls == "good_sit": 
                y = 0
            else:
                y = 1
            data.append({
                'x': img,
                'y': y
            })
    random.shuffle(data)
    X = [d['x'] for d in data]
    Y = [[d['y']] for d in data]

    # 检测模型
    file_model = "posenet_mobilenet_v1_100_257x257_multi_kpt_stripped.tflite"

    interpreter = tf.lite.Interpreter(model_path=file_model)
    interpreter.allocate_tensors()

    # 获取输入、输出的数据的信息
    input_details = interpreter.get_input_details()
    #print('input_details\n',input_details)
    output_details = interpreter.get_output_details()
    #print('output_details',output_details)


    # 获取PosNet 要求输入图像的高和宽
    height = input_details[0]['shape'][1]
    width = input_details[0]['shape'][2]

    key_data = [[]]
    for img in X:
        # 获取图像帧的尺寸
        imH,imW,_ = np.shape(img)

        # 适当缩放
        img = cv2.resize(img,(int(imW*0.5),int(imH*0.5)))

        # 获取图像帧的尺寸
        imH,imW,_ = np.shape(img)

        # BGR 转RGB
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # 尺寸缩放适应PosNet 网络输入要求
        img_resized = cv2.resize(img_rgb, (width, height))

        # 维度扩张适应网络输入要求
        input_data = np.expand_dims(img_resized, axis=0)

        # 尺度缩放 变为 -1~+1
        input_data = (np.float32(input_data) - 128.0)/128.0

        # 数据输入网络
        interpreter.set_tensor(input_details[0]['index'],input_data)

        # 进行关键点检测
        interpreter.invoke()

        # 获取hotmat
        hotmaps = interpreter.get_tensor(output_details[0]['index'])[0] # Bounding box coordinates of detected objects

        # 获取偏移量
        offsets = interpreter.get_tensor(output_details[1]['index'])[0] # Class index of detected objects

        # 获取hotmat的 宽 高 以及关键的数目
        h_output,w_output, n_KeyPoints= np.shape(hotmaps)

        # 存储关键点的具体位置
        keypoints =[]

        # 关键点的置信度
        score_ = []
        for i in range(11):
            # 遍历每一张hotmap
            hotmap = hotmaps[:,:,i]

            # 获取最大值 和最大值的位置
            max_index = np.where(hotmap==np.max(hotmap))
            max_val   = np.max(hotmap)

            # 获取y，x偏移量 前n_KeyPoints张图是y的偏移 后n_KeyPoints张图是x的偏移
            offset_y = offsets[max_index[0],max_index[1],i]
            offset_x = offsets[max_index[0],max_index[1],i+n_KeyPoints]

            # 计算在posnet输入图像中具体的坐标
            pos_y = max_index[0]/(h_output-1)*height + offset_y
            pos_x = max_index[1]/(w_output-1)*width + offset_x

            # 计算在源图像中的坐标
            pos_y = pos_y/(height-1)*imH
            pos_x = pos_x/(width-1)*imW

            #算置信度
            score = 1.0/(1.0+np.exp(-max_val))

            # 取整获得keypoints的位置
            if score>0.2:
                key_data[-1].append(int(round(pos_x[0])))
                key_data[-1].append(int(round(pos_y[0])))
            else:
                key_data[-1].append(-1)
                key_data[-1].append(-1)
            #score_.append(score)

        #print(keypoints[:11], score_)   
        #key_data.append(keypoints[:22])
        key_data.append([])
    key_data = key_data[:-1]

    X_train = np.array(key_data[:-25])
    Y_train = np.array(Y[:-25])
    X_test  = np.array(key_data[-25:])
    Y_test  = np.array(Y[-25:])

    assert len(X_train) == len(Y_train)
    assert len(X_test)  == len(Y_test)

    model = RandomForestClassifier().fit(X_train, Y_train)
    a = model.score(X_test, Y_test)
    total_score += a
    print(a,seed)
    final_score.append((a,seed))
print(final_score)
print(total_score/20)

0.96 0
0.96 1
0.88 2
0.96 3
0.88 4
0.84 5
0.96 6
0.88 7
0.92 8
0.92 9
0.92 10
0.88 11
0.88 12
0.96 13
0.96 14
1.0 15
1.0 16
0.92 17
1.0 18
1.0 19
[(0.96, 0), (0.96, 1), (0.88, 2), (0.96, 3), (0.88, 4), (0.84, 5), (0.96, 6), (0.88, 7), (0.92, 8), (0.92, 9), (0.92, 10), (0.88, 11), (0.88, 12), (0.96, 13), (0.96, 14), (1.0, 15), (1.0, 16), (0.92, 17), (1.0, 18), (1.0, 19)]
0.9340000000000002
